In [8]:
import trimesh
import numpy as np
from io_utils import stdout_redirected
import time
from dvidutils import encode_faces_to_custom_drc_bytes
import dask
import utils

from collections import namedtuple 

Fragment = namedtuple('Fragment', ['draco_bytes', 'position','offset'])

def get_face_indices_in_range(mesh, face_mins, stop):
    max_edge_length = np.max(mesh.edges_unique_length)
    rows = np.where(face_mins[:,0]<stop+max_edge_length)
    return rows[0]

def renumber_vertex_indices(faces, vertex_indices_in_range):
    def renumber_indicies(a, val_old, val_new):
        arr = np.empty(a.max()+1, dtype=val_new.dtype)
        arr[val_old] = val_new
        return arr[a]
    
    faces = np.reshape(faces,-1)
    faces=renumber_indicies(faces,vertex_indices_in_range,np.arange(len(vertex_indices_in_range)))

    return np.reshape(faces,(-1,3))

@dask.delayed
def generate_mesh_decomposition(v,f,box_size, start_fragment, end_fragment, x_start, x_end):
    # Scale our coordinates.

    nyz, nxz, nxy = np.eye(3)
    t = time.time()
    total_blocks = 0
    sz = 0

    fragments = []
    for x in range(x_start, x_end):
        vx, fx = trimesh.intersections.slice_faces_plane(
            v, f, plane_normal=-nyz, plane_origin=nyz*(x+1)*box_size)

        for y in range(start_fragment[1], end_fragment[1]):
            vy, fy = trimesh.intersections.slice_faces_plane(
                vx, fx, plane_normal=-nxz, plane_origin=nxz*(y+1)*box_size)

            for z in range(start_fragment[2], end_fragment[2]):
                vz, fz = trimesh.intersections.slice_faces_plane(
                    vy, fy, plane_normal=-nxy, plane_origin=nxy*(z+1)*box_size)

                if len(vz) > 0:
                    normals = np.zeros(np.shape(vz))
                    draco_bytes = encode_faces_to_custom_drc_bytes(
                        vz, normals, fz, np.asarray(3*[box_size]), np.asarray([x,y,z])*box_size, position_quantization_bits = 10)
                    
                    if len(draco_bytes)>12:
                        fragment = Fragment(draco_bytes, [x, y, z], len(draco_bytes))   
                        fragments.append(fragment)

                vy, fy = trimesh.intersections.slice_faces_plane(
                    vy, fy, plane_normal=nxy, plane_origin=nxy*(z+1)*box_size)

            vx, fx = trimesh.intersections.slice_faces_plane(
                vx, fx, plane_normal=nxz, plane_origin=nxz*(y+1)*box_size)
        
        v, f = trimesh.intersections.slice_faces_plane(
            v, f, plane_normal=nyz, plane_origin=nyz*(x+1)*box_size)

    return fragments

In [9]:
mesh = trimesh.load(
        "/groups/cosem/cosem/ackermand/meshesForWebsite/res1decimation0p1/jrc_hela-1/er_seg/17.obj")
print("read")
print(np.min(mesh.vertices, axis=0))
print(np.max(mesh.vertices, axis=0))

read
[3.49597e+03 3.73832e+00 2.67659e+04]
[27938.8   5679.44 59501.7 ]


In [10]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=4,
                n_workers=16)


/groups/scicompsoft/home/ackermand/miniconda3/envs/simpler_multires/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38049 instead
  f"Port {expected} is already in use.\n"


In [11]:
import dask
results = []
v = mesh.vertices
f = mesh.faces
#face_maxs = np.max(mesh.vertices[mesh.faces],axis=1)

nyz, nxz, nxy = np.eye(3)
box_size = 64*4
num_workers = 16
start_fragment = np.min(mesh.vertices, axis=0).astype(int) // box_size - 1
end_fragment =  np.max(mesh.vertices, axis=0).astype(int) // box_size + 1

x_stride = int(np.ceil(1.0*(end_fragment[0]-start_fragment[0])/num_workers))
for x in range(start_fragment[0], end_fragment[0]+1,x_stride):
    vx, fx = trimesh.intersections.slice_faces_plane(
        v, f, plane_normal=-nyz, plane_origin=nyz*(x+x_stride)*box_size)

    results.append( generate_mesh_decomposition(vx, fx, box_size, start_fragment, end_fragment, x, x+x_stride) )

    v, f = trimesh.intersections.slice_faces_plane(
        v, f, plane_normal=nyz, plane_origin=nyz*(x+x_stride)*box_size)



In [12]:
dask_results = dask.compute(* results)


/groups/scicompsoft/home/ackermand/miniconda3/envs/simpler_multires/lib/python3.7/site-packages/distributed/worker.py:3801: UserWarning: Large object of size 47.30 MiB detected in task graph: 
  (array([[13107.2       ,  4374.88      , 34502.2   ...  233]), 47, 54)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s)


In [13]:
fragments = [fragment for fragments in dask_results for fragment in fragments]
utils.write_files("test/simpler_multires", "17", fragments, 0, [0], np.asarray([box_size,box_size,box_size]))